In [7]:
import pickle
import time
from typing import List
import datetime as dt

import numpy as np
from tqdm import tqdm
import plotly.graph_objects as go
import plotly.express as px

from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis, get_pauli_basis, get_comp_basis
from quara.objects.povm import (
    Povm,
    get_x_measurement,
    get_y_measurement,
    get_z_measurement,
)
from quara.objects.qoperation import QOperation
from quara.objects.state import State, get_x0_1q,  get_x1_1q, get_y0_1q, get_y1_1q, get_z0_1q, get_z1_1q
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.standard_povmt import StandardPovmt
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import ProjectedLinearEstimator

from quara.protocol.qtomography.standard.standard_qtomography_estimator import (
    StandardQTomographyEstimator,
    StandardQTomographyEstimationResult,
)

In [8]:
from quara.data_analysis import data_analysis, physicality_violation_check

In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
def calc_estimate(
    tester_objects: List[State],
    true_object: Povm,
    measurement_n: int,
    num_data: List[int],
    iteration: int,
    estimator=StandardQTomographyEstimator,
    on_para_eq_constraint: bool = True,
) -> StandardQTomographyEstimationResult:

    povmt = StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=on_para_eq_constraint)

    # generate empi dists and calc estimate
    results = []
    for ite in tqdm(range(iteration)):
        empi_dists_seq = povmt.generate_empi_dists_sequence(true_object, num_data)
        result = estimator.calc_estimate_sequence(
            qtomography=povmt, empi_dists_sequence=empi_dists_seq, is_computation_time_required=True
        )

        # info = {
        #    "iteration": ite + 1,
        #     "data": empi_dists_seq,
        #     "estimated_var_sequence": result.estimated_var_sequence,
        #     "computation_times": result.computation_times,
        # }
        # print(info)
        results.append(result)

    identify = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
    with open(f"povmt_data/result_{identify}.pkl", "wb") as f:
        pickle.dump(results, f)
    return results

# Physicality Violation

In [12]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

# |+><+|
state_x0 = get_x0_1q(c_sys)
# |+i><+i|
state_y0 = get_y0_1q(c_sys)
# |0><0|
state_z0 = get_z0_1q(c_sys)
# |1><1|
state_z1 = get_z1_1q(c_sys)
states = [state_x0, state_y0, state_z0, state_z1]

In [13]:
a0, a1, a2, a3 = 1, 1/np.sqrt(2), 0, 1/np.sqrt(2)

m1 = (1/np.sqrt(2)) * np.array([a0, a1, a2, a3])
m2 = (1/np.sqrt(2)) * np.array([2-a0, -a1, -a2, -a3])

true_object = Povm(vecs=[m1, m2], c_sys=c_sys)

In [20]:
# parameter
num_data = [1000]
iterations = 100
measurement_n = len(true_object.vecs)  # 測定値の数

estimator = LinearEstimator()
proj_estimator = ProjectedLinearEstimator()

start = time.time()

param_affine_est_linear = calc_estimate(states, true_object, measurement_n, num_data, iterations, estimator, on_para_eq_constraint=True)
param_linear_est_linear = calc_estimate(states, true_object, measurement_n, num_data, iterations, estimator, on_para_eq_constraint=False)
p_param_affine_est_linear = calc_estimate(states, true_object, measurement_n, num_data, iterations, proj_estimator, on_para_eq_constraint=True)
p_param_linear_est_linear = calc_estimate(states, true_object, measurement_n, num_data, iterations, proj_estimator, on_para_eq_constraint=False)

elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time/60) + "[min]")

  0%|          | 0/100 [00:00<?, ?it/s]/Users/tomoko/project/rcast/workspace/quara/quara/objects/povm.py:245: ComplexWarning: Casting complex values to real discards the imaginary part
  new_vec = np.array(new_vec, dtype=np.float64)
100%|██████████| 100/100 [00:03<00:00, 25.97it/s]

elapsed_time:0.23140121301015218[min]


## LinearEstimator
### on_para_eq_constraint = True

In [31]:
sqrt_d =np.sqrt(m1+m2)
sqrt_d

array([1.18920712, 0.        , 0.        , 0.        ])

In [32]:
estimated_qobjects = [result.estimated_qoperation for result in param_affine_est_linear]
sum_vecs = None

for est in tqdm(estimated_qobjects):
    if sum_vecs is not None:
        sum_vecs = np.vstack([sum_vecs, sum(est.vecs)])
    else:
        sum_vecs = sum(est.vecs)
        
sum_vecs = sum_vecs.T

100%|██████████| 100/100 [00:00<00:00, 16820.28it/s]


In [33]:
vlines_list = [sqrt_d, 0, 0, 0]
for i, value_list in enumerate(sum_vecs):
    fig = physicality_violation_check.make_prob_dist_histogram(value_list, bin_size=0.1, annotation_vlines=[vlines_list[i]])
    title = f"α={i}"
    fig.update_layout(title=title)  # TODO
    fig.show()

### on_para_eq_constraint = False

In [75]:
estimated_qobjects = [result.estimated_qoperation for result in param_linear_est_linear]
eigenvalues_list = None
minus_eigs_list_0 = []
minus_eigs_list_1 = []

for est in tqdm(estimated_qobjects):
    eigenvalues = est.calc_eigenvalues()
    sorted_eigenvalues = []
    for eigs in eigenvalues:
        eigs = sorted(eigs, reverse=True)
        sorted_eigenvalues.append(eigs)

    minus_eigs = []
    
    # TODO: 虚部が10**(-13)より大きい場合はwarningを出す
    minus_eigs_list_0.append(sum([e.real for e in sorted_eigenvalues[0] if e.real < 0]))
    minus_eigs_list_1.append(sum([e.real for e in sorted_eigenvalues[1] if e.real < 0]))                                      
        
len(minus_eigs_list_0)

100%|██████████| 100/100 [00:00<00:00, 2073.95it/s]


100

In [76]:
for i, value_list in enumerate([minus_eigs_list_0, minus_eigs_list_1]):
    fig = physicality_violation_check.make_prob_dist_histogram(value_list, bin_size=0.001, annotation_vlines=[0])
    title = f"各測定値の負の値の固有値の総和の頻度分布 x={i}"
    fig.update_layout(title=title)  # TODO
    fig.show()

In [78]:
eigenvalues_list = None
eigs_list_0 = []
eigs_list_1 = []

for est in tqdm(estimated_qobjects):
    eigenvalues = est.calc_eigenvalues()
    minus_eigs = []
    
    # TODO: 虚部が10**(-13)より大きい場合はwarningを出す
    sorted_eigenvalues = []
    for eigs in eigenvalues:  # 各測定値の固有値
        eigs = sorted(eigs, reverse=True)
        eigs_real = [eig.real for eig in eigs]
        sorted_eigenvalues.append(eigs_real)

    eigs_list_0.append(sorted_eigenvalues[0])  # 測定値1
    eigs_list_1.append(sorted_eigenvalues[1])  # 測定値2

eigs_list_0 = np.array(eigs_list_0).T
eigs_list_1 = np.array(eigs_list_1).T

100%|██████████| 100/100 [00:00<00:00, 2093.47it/s]


In [79]:
for x, eigs_list in tqdm(enumerate([eigs_list_0, eigs_list_1])):  # 各測定値
    for i, value_list  in tqdm(enumerate(eigs_list)):  # 各固有値
        n_rep = iterations
        n = num_data[0]
        title = f"λ x={x}, i={i}"
        title += "<br>"
        title += f"N={n}, Nrep={n_rep}"
        
        fig = physicality_violation_check.make_prob_dist_histogram(value_list, bin_size=0.001)
        fig.update_layout(title=title)  # TODO
        fig.show()

0it [00:00, ?it/s]
0it [00:00, ?it/s]

2it [00:00, 23.10it/s]

0it [00:00, ?it/s]

2it [00:00, 22.24it/s]
2it [00:00, 10.00it/s]


In [42]:
total_vecs_0 = []
total_vecs_1 = []
for est in tqdm(estimated_qobjects):
    total_vecs_0.append(est.vecs[0])
    total_vecs_1.append(est.vecs[1])

100%|██████████| 100/100 [00:00<00:00, 90336.08it/s]


In [43]:
sum(total_vecs_0) / 1000

array([ 0.07072411,  0.05003841, -0.00035285,  0.05004124])

In [44]:
sum(total_vecs_1) / 1000

array([ 0.07069724, -0.05003841,  0.00035285, -0.05004124])

In [45]:
true_object.vecs

(array([0.70710678, 0.5       , 0.        , 0.5       ]),
 array([ 0.70710678, -0.5       ,  0.        , -0.5       ]))

## ProjectedLinearEstimator
### on_para_eq_constraint = True

In [51]:
estimated_qobjects = [result.estimated_qoperation for result in p_param_affine_est_linear]
sum_vecs = None

for est in tqdm(estimated_qobjects):
    if sum_vecs is not None:
        sum_vecs = np.vstack([sum_vecs, sum(est.vecs)])
    else:
        sum_vecs = sum(est.vecs)
        
sum_vecs = sum_vecs.T

100%|██████████| 100/100 [00:00<00:00, 14278.00it/s]


In [52]:
vlines_list = [sqrt_d, 0, 0, 0]
for i, value_list in enumerate(sum_vecs):
    fig = physicality_violation_check.make_prob_dist_histogram(value_list, bin_size=0.1, annotation_vlines=[vlines_list[i]])
    title = f"α={i}"
    fig.update_layout(title=title)  # TODO
    fig.show()

### on_para_eq_constraint = False

In [80]:
estimated_qobjects = [result.estimated_qoperation for result in p_param_linear_est_linear]
eigenvalues_list = None
minus_eigs_list_0 = []
minus_eigs_list_1 = []

for est in tqdm(estimated_qobjects):
    eigenvalues = est.calc_eigenvalues()
    sorted_eigenvalues = []
    for eigs in eigenvalues:
        eigs = sorted(eigs, reverse=True)
        sorted_eigenvalues.append(eigs)

    minus_eigs = []
    
    # TODO: 虚部が10**(-13)より大きい場合はwarningを出す
    minus_eigs_list_0.append(sum([e.real for e in sorted_eigenvalues[0] if e.real < 0]))
    minus_eigs_list_1.append(sum([e.real for e in sorted_eigenvalues[1] if e.real < 0]))                                      
        
len(minus_eigs_list_0)

100%|██████████| 100/100 [00:00<00:00, 596.39it/s]


100

In [91]:
[v for v in minus_eigs_list_1 if v > 0]

[]

In [81]:
for i, value_list in enumerate([minus_eigs_list_0, minus_eigs_list_1]):
    fig = physicality_violation_check.make_prob_dist_histogram(value_list, bin_size=0.001, annotation_vlines=[0])
    title = f"各測定値の負の値の固有値の総和の頻度分布 x={i}"
    fig.update_layout(title=title)  # TODO
    fig.show()

In [82]:
eigenvalues_list = None
eigs_list_0 = []
eigs_list_1 = []

for est in tqdm(estimated_qobjects):
    eigenvalues = est.calc_eigenvalues()
    minus_eigs = []
    
    # TODO: 虚部が10**(-13)より大きい場合はwarningを出す
    sorted_eigenvalues = []
    for eigs in eigenvalues:  # 各測定値の固有値
        eigs = sorted(eigs, reverse=True)
        eigs_real = [eig.real for eig in eigs]
        sorted_eigenvalues.append(eigs_real)

    eigs_list_0.append(sorted_eigenvalues[0])  # 測定値1
    eigs_list_1.append(sorted_eigenvalues[1])  # 測定値2

eigs_list_0 = np.array(eigs_list_0).T
eigs_list_1 = np.array(eigs_list_1).T

100%|██████████| 100/100 [00:00<00:00, 1848.57it/s]


In [83]:
for x, eigs_list in tqdm(enumerate([eigs_list_0, eigs_list_1])):
    for i, value_list  in tqdm(enumerate(eigs_list)):
        n_rep = iterations
        n = num_data[0]
        title = f"λ x={x}, i={i}"
        title += "<br>"
        title += f"N={n}, Nrep={n_rep}"
        
        fig = physicality_violation_check.make_prob_dist_histogram(value_list, bin_size=0.001)
        fig.update_layout(title=title)  # TODO
        fig.show()

0it [00:00, ?it/s]
0it [00:00, ?it/s]


2it [00:00, 18.53it/s]
1it [00:00,  8.41it/s]
0it [00:00, ?it/s]

2it [00:00, 21.36it/s]
2it [00:00,  8.56it/s]


In [86]:
estimated_qobjects = [result.estimated_qoperation for result in param_linear_est_linear]
len([est for est in estimated_qobjects if not est.is_physical()])

72

In [87]:
estimated_qobjects = [result.estimated_qoperation for result in p_param_linear_est_linear]
len([est for est in estimated_qobjects if not est.is_physical()])

71

In [88]:
estimated_qobjects = [result.estimated_qoperation for result in param_affine_est_linear]
len([est for est in estimated_qobjects if not est.is_physical()])

73

In [89]:
estimated_qobjects = [result.estimated_qoperation for result in p_param_affine_est_linear]
len([est for est in estimated_qobjects if not est.is_physical()])

72